
Description
Changes comparing V1:

Adding new features:

df['u_in_first'] = df.groupby(['breath_id'])['u_in'].transform('first')
df['u_in_last'] = df.groupby(['breath_id'])['u_in'].transform('last')

df.loc[train['time_step'] == 0, 'u_in_diff'] = 0
df.loc[train['time_step'] == 0, 'u_out_diff'] = 0

Using Rescaling laying for discrete output

Ensemble folds with median
Ensembble folds with median and round predictions

Results:

Median: 0.150
Median and round predictions: 0.159

V2:
Attention

In [1]:
import numpy as np
import pandas as pd

import optuna
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.optimizers.schedules import ExponentialDecay

from sklearn.metrics import mean_absolute_error as mae
from sklearn.preprocessing import RobustScaler, normalize
from sklearn.model_selection import train_test_split, GroupKFold, KFold

from IPython.display import display

In [2]:
DEBUG = False

train = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
test = pd.read_csv('../input/ventilator-pressure-prediction/test.csv')
submission = pd.read_csv('../input/ventilator-pressure-prediction/sample_submission.csv')

if DEBUG:
    train = train[:80*1000]

In [3]:
all_pressure = np.sort(train['pressure'].unique())
pressure_min =  all_pressure[0].item()
pressure_max = all_pressure[-1].item()
pressure_step = (all_pressure[1] - all_pressure[0]).item()

In [4]:
display(pressure_min)
display(pressure_max)
display(pressure_step)

-1.895744294564641

64.8209917386395

0.07030214545121005

## Engineer Features

## From [Ventilator: Feature engineering](https://www.kaggle.com/mistag/ventilator-feature-engineering)

In [5]:
#Feature engineering
def add_features(df):
    #time_step*u_in
    df['area'] = df['time_step'] * df['u_in']
    df['area'] = df.groupby('breath_id')['area'].cumsum()
    
    #sum of u_in
    df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()
    
    #shift +1 -1 +3 -3
    df['u_in_lag1'] = df.groupby('breath_id')['u_in'].shift(1)
    df['u_out_lag1'] = df.groupby('breath_id')['u_out'].shift(1)
    df['u_in_lag_back1'] = df.groupby('breath_id')['u_in'].shift(-1)
    df['u_out_lag_back1'] = df.groupby('breath_id')['u_out'].shift(-1)
    df['u_in_lag2'] = df.groupby('breath_id')['u_in'].shift(2)
    df['u_out_lag2'] = df.groupby('breath_id')['u_out'].shift(2)
    df['u_in_lag_back2'] = df.groupby('breath_id')['u_in'].shift(-2)
    df['u_out_lag_back2'] = df.groupby('breath_id')['u_out'].shift(-2)
    df['u_in_lag3'] = df.groupby('breath_id')['u_in'].shift(3)
    df['u_out_lag3'] = df.groupby('breath_id')['u_out'].shift(3)
    df['u_in_lag_back3'] = df.groupby('breath_id')['u_in'].shift(-3)
    df['u_out_lag_back3'] = df.groupby('breath_id')['u_out'].shift(-3)
    df['u_in_lag4'] = df.groupby('breath_id')['u_in'].shift(4)
    df['u_out_lag4'] = df.groupby('breath_id')['u_out'].shift(4)
    df['u_in_lag_back4'] = df.groupby('breath_id')['u_in'].shift(-4)
    df['u_out_lag_back4'] = df.groupby('breath_id')['u_out'].shift(-4)
    df = df.fillna(0)
         
    df['u_in_first'] = df.groupby(['breath_id'])['u_in'].transform('first')
    df['u_in_last'] = df.groupby(['breath_id'])['u_in'].transform('last')
    
    # max value of u_in and u_out for each breath
    df['breath_id__u_in__max'] = df.groupby(['breath_id'])['u_in'].transform('max')
    df['breath_id__u_out__max'] = df.groupby(['breath_id'])['u_out'].transform('max')
   
    # difference between consequitive values
    df['u_in_diff1'] = df['u_in'] - df['u_in_lag1']
    df['u_out_diff1'] = df['u_out'] - df['u_out_lag1']
    df['u_in_diff2'] = df['u_in'] - df['u_in_lag2']
    df['u_out_diff2'] = df['u_out'] - df['u_out_lag2']
    
    df.loc[df['time_step'] == 0, 'u_in_diff'] = 0
    df.loc[df['time_step'] == 0, 'u_out_diff'] = 0
    
    # difference between the current value of u_in and the max value within the breath
    df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
    # difference between the current value of u_in and the mean value within the breath
    df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']
    
    # difference between consequitive values
    df['u_in_diff3'] = df['u_in'] - df['u_in_lag3']
    df['u_out_diff3'] = df['u_out'] - df['u_out_lag3']
    df['u_in_diff4'] = df['u_in'] - df['u_in_lag4']
    df['u_out_diff4'] = df['u_out'] - df['u_out_lag4']
    
    #u_in*u_out
    df['cross']= df['u_in']*df['u_out']
    
    #time_step*u_out
    df['cross2']= df['time_step']*df['u_out']
    
    df['R'] = df['R'].astype(str)
    df['C'] = df['C'].astype(str)
    df['R__C'] = df["R"].astype(str) + '__' + df["C"].astype(str)
    #one hot encoding
    df = pd.get_dummies(df)
    
    df = df.fillna(0)
    return df

train = add_features(train)
test = add_features(test)

In [6]:
train.shape
test.shape

(4024000, 56)

In [7]:
check_train_nan = train.isnull().sum().sum()
check_test_nan = test.isnull().sum().sum()

print(check_train_nan)
print(check_test_nan)

0
0


In [8]:
train.head(3)

,id,breath_id,time_step,u_in,u_out,pressure,area,u_in_cumsum,u_in_lag1,u_out_lag1,...,C_50,R__C_20__10,R__C_20__20,R__C_20__50,R__C_50__10,R__C_50__20,R__C_50__50,R__C_5__10,R__C_5__20,R__C_5__50
0,1,1,0.000000,0.083334,0,5.837492,0.000000,0.083334,0.000000,0.0,...,1,0,0,1,0,0,0,0,0,0
1,2,1,0.033652,18.383041,0,5.907794,0.618632,18.466375,0.083334,0.0,...,1,0,0,1,0,0,0,0,0,0
2,3,1,0.067514,22.509278,0,7.876254,2.138333,40.975653,18.383041,0.0,...,1,0,0,1,0,0,0,0,0,0


In [9]:
targets = train[['pressure']].to_numpy().reshape(-1, 80)
train.drop(['pressure', 'id', 'breath_id'], axis=1, inplace=True)
test = test.drop(['id', 'breath_id'], axis=1)

In [10]:
#Normalise the dataset
RS = RobustScaler()
train = RS.fit_transform(train)
test = RS.transform(test)

In [11]:
#Reshape group to 80 timesteps for each breath ID
train = train.reshape(-1, 80, train.shape[-1])
test = test.reshape(-1, 80, train.shape[-1])

## Model

## From [Rescaling layer for discrete output in TensorFlow](https://www.kaggle.com/lucamassaron/rescaling-layer-for-discrete-output-in-tensorflow)

Please notice the custom rounding round_with_gradients function since tf.round has no gradients and it won't be differentiable.

In [12]:
@tf.custom_gradient
def round_with_gradients(x):
    def grad(dy):
        return dy
    return tf.round(x), grad

class ScaleLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(ScaleLayer, self).__init__()
        self.min = tf.constant(pressure_min, dtype=np.float32)
        self.max = tf.constant(pressure_max, dtype=np.float32)
        self.step = tf.constant(pressure_step, dtype=np.float32)

    def call(self, inputs):
        steps = tf.math.divide(tf.math.add(inputs, -self.min), self.step)
        int_steps = round_with_gradients(steps)
        rescaled_steps = tf.math.add(tf.math.multiply(int_steps, self.step), self.min)
        clipped = tf.clip_by_value(rescaled_steps, self.min, self.max)
        return clipped

In [13]:
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K

class Attention(Layer):
    
    def __init__(self, return_sequences=True):
        self.return_sequences = return_sequences
        super(Attention,self).__init__()
        
    def build(self, input_shape):
        
        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1],1),
                               initializer="normal")
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1),
                               initializer="zeros")
        
        super(Attention,self).build(input_shape)
        
    def call(self, x):
        
        e = K.tanh(K.dot(x,self.W)+self.b)
        a = K.softmax(e, axis=1)
        output = x*a
        
        if self.return_sequences:
            return output
        
        return K.sum(output, axis=1)

    def get_config(self):
        return super(Attention, self).get_config()

In [14]:
def BiLSTM_model():
    inputs = keras.layers.Input(shape=train.shape[-2:])
    x = inputs
    #bidirectional LSTM 1024->512->256->128
    x = keras.layers.Bidirectional(keras.layers.LSTM(1024, return_sequences=True))(x)
    attend_x = Attention(return_sequences=True)(x)
    x = keras.layers.Concatenate()([x, attend_x])
    x = keras.layers.Bidirectional(keras.layers.LSTM(512, return_sequences=True))(x)
    attend_x = Attention(return_sequences=True)(x)
    x = keras.layers.Concatenate()([x, attend_x])
    x = keras.layers.Bidirectional(keras.layers.LSTM(256, return_sequences=True))(x)
    attend_x = Attention(return_sequences=True)(x)
    x = keras.layers.Concatenate()([x, attend_x])
    x = keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True))(x)
    attend_x = Attention(return_sequences=True)(x)
    x = keras.layers.Concatenate()([x, attend_x])
    x = keras.layers.Dense(128, activation='selu')(x)
    # keras.layers.Dropout(0.1)
    outputs = keras.layers.Dense(1)(x)
    outputs = ScaleLayer()(outputs)

    model  = keras.Model(inputs=inputs, outputs=outputs)
    model.summary()
    model.compile(optimizer="adam", loss="mae")
    return model

In [15]:
EPOCH = 300
BATCH_SIZE = 1024
NUM_FOLDS = 10

# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

#GPU init
#gpu_strategy = tf.distribute.get_strategy()

#with gpu_strategy.scope():
with tpu_strategy.scope():
    kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=2021)
    test_preds = []
    
        
    for fold, (train_idx, test_idx) in enumerate(kf.split(train, targets)):
        print('-'*15, '>', f'Fold {fold+1}', '<', '-'*15)
        X_train, X_valid = train[train_idx], train[test_idx]
        y_train, y_valid = targets[train_idx], targets[test_idx]

        model = BiLSTM_model()
        
        scheduler = ExponentialDecay(1e-3, 40*((len(train)*0.8)/BATCH_SIZE), 1e-5)
        lr = LearningRateScheduler(scheduler, verbose=1)
        
        #lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=10, verbose=1)
        #lr = WarmupExponentialDecay(lr_base=1e-3, decay=1e-5, warmup_epochs=30)
        es = EarlyStopping(monitor="val_loss", patience=60, verbose=1, mode="min", restore_best_weights=True)
    
        checkpoint_filepath = f"folds{fold}.hdf5"
        sv = keras.callbacks.ModelCheckpoint(
            checkpoint_filepath, monitor='val_loss', verbose=1, save_best_only=True,
            save_weights_only=False, mode='auto', save_freq='epoch',
            options=None
        )

        model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=EPOCH, batch_size=BATCH_SIZE, callbacks=[lr, es, sv])
        #model.save(f'Fold{fold+1} RNN Weights')
        test_preds.append(model.predict(test).squeeze().reshape(-1, 1).squeeze())

--------------- > Fold 1 < ---------------
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 80, 54)]     0                                            
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 80, 2048)     8839168     input_1[0][0]                    
__________________________________________________________________________________________________
attention (Attention)           (None, 80, 2048)     2128        bidirectional[0][0]              
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 80, 4096)     0           bidirectional[0][0]              
                                                   

## Median method from [Chris Deotte](https://www.kaggle.com/cdeotte/ensemble-folds-with-median-0-153)

In [16]:
submission["pressure"] = sum(test_preds)/NUM_FOLDS
submission.to_csv('submission.csv', index=False)

# ENSEMBLE FOLDS WITH MEDIAN
#取中位数
submission["pressure"] = np.median(np.vstack(test_preds),axis=0)
submission.to_csv('submission_median.csv', index=False)


# ENSEMBLE FOLDS WITH MEDIAN AND ROUND PREDICTIONS
submission["pressure"] =\
    np.round( (submission.pressure - pressure_min)/pressure_step ) * pressure_step + pressure_min
submission.pressure = np.clip(submission.pressure, pressure_min, pressure_max)
submission.to_csv('submission_median_round.csv', index=False)